<a href="https://colab.research.google.com/github/Pingfaisnuli/ICI-s_chatbot/blob/main/%5BIntroAI%5D_Final_Chatgpt_bot_BRI_llama_index.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
# Reference 
# https://llamahub.ai/l/file
# https://gpt-index.readthedocs.io/en/stable/guides/primer/usage_pattern.html
# https://github.com/jerryjliu/llama_index/issues/1900
# https://github.com/jerryjliu/llama_index

In [ ]:
#https://zhuanlan.zhihu.com/p/613155165
#https://www.wbolt.com/building-a-chatbot-based-on-documents-with-gpt.html

!pip install llama-index
from llama_index import GPTVectorStoreIndex, SimpleDirectoryReader, Document, LLMPredictor , PromptHelper, ServiceContext, GPTListIndex

In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import os
os.environ["OPENAI_API_KEY"] = 'Fill in your API'
documents = SimpleDirectoryReader('/content/drive/MyDrive/IntroAI/FAQs.txt').load_data()

In [ ]:
from llama_index.node_parser import SimpleNodeParser

parser = SimpleNodeParser()

nodes = parser.get_nodes_from_documents(documents)

In [ ]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex.from_documents(documents)

In [ ]:
from llama_index import StorageContext

storage_context = StorageContext.from_defaults()
storage_context.docstore.add_documents(nodes)

index1 = GPTVectorStoreIndex(nodes, storage_context=storage_context)
index2 = GPTListIndex(nodes, storage_context=storage_context)

In [ ]:
from llama_index import GPTVectorStoreIndex

index = GPTVectorStoreIndex([])
for doc in documents:
    index.insert(doc)

In [ ]:
from llama_index import LLMPredictor, GPTVectorStoreIndex, PromptHelper, ServiceContext
from langchain import OpenAI

# define LLM
llm_predictor = LLMPredictor(llm=OpenAI(temperature=0, model_name="gpt-3.5-turbo"))

# define prompt helper
# set maximum input size
max_input_size = 4096
# set number of output tokens
num_output = 256
# set maximum chunk overlap (e.g., 20% overlap)
max_chunk_overlap = 0.2
prompt_helper = PromptHelper(max_input_size, num_output, max_chunk_overlap)

service_context = ServiceContext.from_defaults(llm_predictor=llm_predictor, prompt_helper=prompt_helper)

index = GPTVectorStoreIndex.from_documents(
    documents, service_context=service_context
)


from llama_index import (
    GPTVectorStoreIndex,
    ResponseSynthesizer,
)
from llama_index.retrievers import VectorIndexRetriever
from llama_index.query_engine import RetrieverQueryEngine
from llama_index.indices.postprocessor import SimilarityPostprocessor

# configure retriever
retriever = VectorIndexRetriever(
    index=index, 
    similarity_top_k=2,
)

# configure response synthesizer
response_synthesizer = ResponseSynthesizer.from_args(
    node_postprocessors=[
        SimilarityPostprocessor(similarity_cutoff=0.7)
    ]
)

# assemble query engine
query_engine = RetrieverQueryEngine(
    retriever=retriever,
    response_synthesizer=response_synthesizer,
)

# define a list of inappropriate or offensive words and phrases
inappropriate_content = ["bad word", "offensive phrase"]

# get user query
user_query = input("Enter your question: ")

# check if user query contains any inappropriate or offensive content
if any(content in user_query for content in inappropriate_content):
    # inform user that their question is not supported
    print("Sorry, your question is not supported")
else:
    # pass query to query engine
    response = query_engine.query(user_query)
    if not response:
        print("Sorry, I can't answer this")
    else:
        print(response)


In [ ]:
pip install gradio

In [ ]:
import gradio as gr

def answer_question(question):
    # define a list of inappropriate or offensive words and phrases
    inappropriate_content = ["Who are you", "offensive phrase"]
    
    # check if user query contains any inappropriate or offensive content
    if any(content in question for content in inappropriate_content):
        # inform user that their question is not supported
        return "Sorry, your question is not supported"
    else:
        # build index
        index = GPTVectorStoreIndex.from_documents(
        documents, service_context=service_context)

        # configure retriever
        retriever = VectorIndexRetriever(
            index=index, 
            similarity_top_k=2,
        )

        # configure response synthesizer
        response_synthesizer = ResponseSynthesizer.from_args(
            node_postprocessors=[
                SimilarityPostprocessor(similarity_cutoff=0.7)
            ]
        )

        # assemble query engine
        query_engine = RetrieverQueryEngine(
            retriever=retriever,
            response_synthesizer=response_synthesizer,
        )

        # query
        response = query_engine.query(question)
        return response

iface = gr.Interface(fn=answer_question, inputs="text", outputs="text", title="Question Answering System")
iface.launch()